# Police use of force in California

We downloaded two use of force datasets from the [California Department of Justice](https://openjustice.doj.ca.gov/data). The codebook for these variables can be found [here](https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2020-07/URSUS%202019%20Context_07012020.pdf)

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_column', 200)

In [2]:
df = pd.read_csv('raw_data/URSUS_Incident_2019.csv')

In [3]:
df.head()

,INCIDENT_ID,ORI,INCIDENT_DATE_STR,INCIDENT_TIME_STR,CITY,COUNTY,STATE,ZIP_CODE,MULTIPLE_LOCATIONS,ON_K12_CAMPUS,ARREST_MADE,CRIME_REPORT_FILED,CONTACT_REASON,IN_CUSTODY_REASON,NUM_INVOLVED_CIVILIANS,NUM_INVOLVED_OFFICERS,NUM_INVOLVED_AGENCIES
0,URSUS-01-0000-2019-402,CA0010000,1/6/2019,16:20,DUBLIN,ALAMEDA,CA,94568,False,False,True,True,in_custody_event,booked_awaiting_trial,1,2,1
1,URSUS-01-0000-2019-481,CA0010000,4/8/2019,10:30,DUBLIN,ALAMEDA,CA,94568,False,False,True,True,in_custody_event,booked_awaiting_trial,1,5,1
2,URSUS-01-0000-2019-684,CA0010000,12/13/2019,22:30,SAN LEANDRO,ALAMEDA,CA,94578,False,False,True,True,call_for_service,NaN,1,2,1
3,URSUS-01-0000-2019-6fc,CA0010000,3/13/2019,16:33,SAN LEANDRO,ALAMEDA,CA,94578,False,False,False,True,call_for_service,NaN,1,3,1
4,URSUS-01-0000-2019-701,CA0010000,11/26/2019,06:00,DUBLIN,ALAMEDA,CA,94568,False,False,True,True,in_custody_event,awaiting_booking,1,3,1


This largely doesn't seem useful without details about the incident itself. It might be possible to cross-reference the incident ID with incident details from another dataset? Let's look at the other dataset.

In [4]:
df2 = pd.read_csv('raw_data/URSUS_Civilian-Officer_2019.csv')

In [5]:
df2.columns
# thats_more_like_it.jpg

Index(['Incident_ID', 'Civilian_Officer', 'ORI', 'Gender', 'Race_Ethnic_Group',
       'Asian_Race', 'HI_Islander_Race', 'Age', 'Injured', 'Injury_Level',
       'Injury_Type', 'Medical_Aid', 'Injury_From_Preexisting_Condition',
       'Received_Force_Location', 'Order_Of_Force_Specified',
       'Order_Of_Force_Str', 'Received_Force', 'Received_Force_Type',
       'DISCHARGE_OF_FIREARM_INDIVIDUAL', 'DISCHARGE_OF_FIREARM_INCIDENT',
       'CIVILIAN_Signs_Impairment_Disability', 'CIVILIAN_Assaulted_Officer',
       'CIVILIAN_Highest_Charge', 'CIVILIAN_Perceived_Armed',
       'CIVILIAN_Perceived_Armed_Weapon', 'CIVILIAN_Firearm_Type',
       'CIVILIAN_Resisted', 'CIVILIAN_Resistance_Type',
       'CIVILIAN_Custody_Status', 'CIVILIAN_Confirmed_Armed',
       'CIVILIAN_Confirmed_Armed_Weapon', 'CIVILIAN_K12_Type',
       'OFFICER_Officer_Used_Force', 'OFFICER_Officer_Used_Force_Reason',
       'OFFICER_On_Duty', 'OFFICER_Dress'],
      dtype='object')

So it looks like df2 has details about the incident, while df1 contains information about the location of the incident. Hopefully the incident IDs match up and we can do a geographical analysis as well.

In [6]:
df2.columns= df2.columns.str.lower()
# no-shift meta

In [7]:
df2.head()

,incident_id,civilian_officer,ori,gender,race_ethnic_group,asian_race,hi_islander_race,age,injured,injury_level,injury_type,medical_aid,injury_from_preexisting_condition,received_force_location,order_of_force_specified,order_of_force_str,received_force,received_force_type,discharge_of_firearm_individual,discharge_of_firearm_incident,civilian_signs_impairment_disability,civilian_assaulted_officer,civilian_highest_charge,civilian_perceived_armed,civilian_perceived_armed_weapon,civilian_firearm_type,civilian_resisted,civilian_resistance_type,civilian_custody_status,civilian_confirmed_armed,civilian_confirmed_armed_weapon,civilian_k12_type,officer_officer_used_force,officer_officer_used_force_reason,officer_on_duty,officer_dress
0,URSUS-01-0000-2019-402,Civilian,CA0010000,male,black,NaN,NaN,18_20,False,NaN,NaN,NaN,False,arms_hands,False,other_control_hold,True,other_control_hold,False,False,none,True,69 OBSTRUCT/RESIST EXEC OFF,False,NaN,NaN,True,assaultive,in_custody_other,False,NaN,NaN,NaN,NaN,NaN,NaN
1,URSUS-01-0000-2019-402,Officer,CA0010000,male,white,NaN,NaN,26_30,True,serious_injury,concussion,treated_at_facility_released,False,head,NaN,NaN,True,civilian_physical,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,overcome_resistance,True,utility
2,URSUS-01-0000-2019-402,Officer,CA0010000,male,white,NaN,NaN,41_45,False,NaN,NaN,NaN,False,NaN,NaN,NaN,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,True,utility
3,URSUS-01-0000-2019-481,Civilian,CA0010000,male,black,NaN,NaN,21_25,True,injury,abrasion_laceration,treated_on_scene,False,"rear_lower_back, arms_hands",False,"other_control_hold, other_physical_contact, el...",True,"other_control_hold, other_physical_contact, el...",False,False,mental_disability,True,69 OBSTRUCT/RESIST EXEC OFF,True,knife_blade_stabbing,NaN,True,assaultive,in_custody_other,False,NaN,NaN,NaN,NaN,NaN,NaN
4,URSUS-01-0000-2019-481,Officer,CA0010000,male,white,NaN,NaN,31_35,True,serious_injury,"bone_fracture, internal_injury",critial_injuries,False,head,NaN,NaN,True,civilian_physical,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,"effect_arrest, overcome_resistance",True,utility


In [8]:
df2.injury_level.unique()

array([nan, 'serious_injury', 'injury', 'death'], dtype=object)

In [9]:
df2.injury_type.unique()

array([nan, 'concussion', 'abrasion_laceration',
       'bone_fracture, internal_injury', 'bone_fracture', 'gunshot_wound',
       'sprain, dislocation', 'unconscious', 'internal_injury', 'sprain',
       'cut', 'abrasion_laceration, cut, bruised',
       'abrasion_laceration, bruised', 'dislocation',
       'gunshot_wound, bruised',
       'bone_fracture, abrasion_laceration, bruised',
       'abrasion_laceration, stabbing_wound, cut',
       'unconscious, bone_fracture', 'bruised', 'bone_fracture, cut',
       'internal_injury, obvious_disfigurement',
       'abrasion_laceration, cut',
       'bone_fracture, internal_injury, bruised',
       'bone_fracture, abrasion_laceration, cut, bruised',
       'abrasion_laceration, gunshot_wound', 'bone_fracture, dislocation',
       'bone_fracture, abrasion_laceration',
       'internal_injury, abrasion_laceration, cut', 'gunshot_wound, cut',
       'internal_injury, dislocation',
       'bone_fracture, internal_injury, abrasion_laceration, br

Multiple injuries on one incident are going to make this difficult to sift through. We can describe the data in terms of severity of injury, perhaps investigate racial disparaties.

In [10]:
df2.medical_aid.unique()

array([nan, 'treated_at_facility_released', 'treated_on_scene',
       'critial_injuries', 'died_in_transit', 'no_medical_aid',
       'precautionary_measure_only', 'other_circumstance'], dtype=object)

In [11]:
df2.received_force_type.unique()

array(['other_control_hold', 'civilian_physical', nan,
       'other_control_hold, other_physical_contact, electronic_device',
       'other_control_hold, other_physical_contact',
       'discharge_of_firearm_hit', 'civilian_physical, civilian_vehicle',
       'k_9_contact', 'discharge_of_firearm_miss', 'threat_of_firearm',
       'civilian_vehicle', 'other_physical_contact',
       'other_dangerous_weapon', 'other_physical_contact, k_9_contact',
       'carotid_hold, other_control_hold, other_physical_contact',
       'civilian_physical, other_dangerous_weapon',
       'blunt_impact_weapon, k_9_contact', 'knife_blade_stabbing',
       'impact_projectile, discharge_of_firearm_hit',
       'blunt_impact_weapon, discharge_of_firearm_hit',
       'impact_projectile, discharge_of_firearm_miss, discharge_of_firearm_hit',
       'other_physical_contact, electronic_device',
       'discharge_of_firearm_hit, k_9_contact',
       'civilian_physical, blunt_impact_weapon',
       'carotid_hold, o

In [12]:
df2.civilian_perceived_armed_weapon.unique()

array([nan, 'knife_blade_stabbing', 'other_dangerous_weapon', 'firearm',
       'unknown', 'firearm, knife_blade_stabbing',
       'knife_blade_stabbing, other_dangerous_weapon',
       'firearm, knife_blade_stabbing, other_dangerous_weapon',
       'other_dangerous_weapon, unknown',
       'firearm, other_dangerous_weapon'], dtype=object)

In [13]:
df2.civilian_resistance_type.unique()

array(['assaultive', nan, 'active_resistance', 'life_threatening',
       'fleeing', 'passive_non_compliance'], dtype=object)